# Titanic Survival Classificaion:


### Required Imports :

In [ ]:
#required libraries:
import pandas as pd;    
import numpy as np;

### Utility Functions:

In [ ]:
#importing training and test data into pandas dataframe
def import_data(path):
    return pd.read_csv(path)



In [ ]:
#utility functions
def print_info(df):
    return df.info()

def print_description(df):
    return df.describe()

def print_head(df, count=5):
    return df.head(count)

### Preprocess both training and testing dataframe :

In [ ]:
def numeric_mapper(df, column_name):
    mapper = {}
    data_list = df[column_name].unique()
    data_list = ['missing' if pd.isnull(x) else x for x in data_list]
    data_list.sort()
    for i in range(0, len(data_list)):
        if data_list[i] == 'missing':
            mapper[np.nan] = 404
        else:    
            mapper[data_list[i]] = i
    return mapper
    
def data_preprocessor(df):
 
    df = (df.rename({'SibSp' : '# of Siblings', 
                                             'Parch': '# of Parents', 
                                             'Sex' : 'Gender',
                                             'Pclass' : 'Class'},
                                            axis = 1)
                                    .drop(['Name', 'Ticket', 'PassengerId'], axis = 1)
                                    .astype({'Gender' : pd.api.types.CategoricalDtype(df['Sex'].unique(), ordered=False), 
                                             'Class' : pd.api.types.CategoricalDtype(df['Pclass'].unique(), ordered=True)})
#                                     .replace({'Embarked' : {np.NaN : 'un-known'}})
                                    .replace({'Gender' : numeric_mapper(df, 'Sex'),
                                              'Embarked' : numeric_mapper(df, 'Embarked'),
                                              'Cabin' : numeric_mapper(df, 'Cabin'),
                                              'Age' : {np.nan : 404}})
                                    
    #                                 .loc[:]
                      )
    return df

### Dealing With Null Values In Attributes :

#### CASE 1 : Removing attributes with null values:

In [ ]:
# In training only 2 attributes contains a major chunk of null data : Age and Cabin
def drop_null_attributes(df):
    df = (df.dropna(axis='columns')
         )
    return df

#### CASE 2: Remove rows with null values:

In [ ]:
def drop_null_rows(df):
    df = (df.dropna(axis='rows')
         )
    return df

#### CASE 3: Replacing null attribute values with mean, median or mode

In [ ]:
# replace nulls with averages 
def replace_null_with_mean(col):
    if col.dtype.name == "category":
        col = col.replace(np.nan,col.mode())
    else:
        col = col.replace(np.nan, col.mean())
    return col

def replace_nulls_phase_1(df):
    df = (df.apply(replace_null_with_mean, axis = 0)
         )
    return df

#### CASE 4: Group attributes with similar values and replace null values with mean or mode values of that specific group

In [ ]:
# Function to return attributes with greatest correlation with the provided attribute
def top_correlations(df,attribute, count = 2):
    correlations_df  = df.corr()
    correlation_attribute = correlations_df[attribute]
    correlation_attribute = correlation_attribute.to_frame()
    correlation_attribute[attribute+'_mod'] = [ x if x > 0 else -1*x for x in correlations_df[attribute]]
    return (correlation_attribute.sort_values(attribute+'_mod', ascending = False)[1: (count+1)].loc[:, attribute])
    

In [ ]:
# get attribute with atleast 1 cell having a null value
def get_attributes_with_nulls(df):
    df_nulls = pd.Series(titanic_train_4.isna().any())
    return df_nulls[df_nulls == True].index.tolist()

### Classification Models :

#### Logistic Regression 

In [ ]:
def create_logistic_regression_model(df):
    from sklearn.model_selection import train_test_split
    from sklearn.linear_model import LogisticRegression

    X_train, X_test, y_train, y_test = train_test_split(df.loc[:, df.columns!= 'Survived'], df['Survived'], test_size = 0.75, random_state = 0)
    clf = LogisticRegression(solver = 'liblinear') #solver : liblinear good for small datasets
    clf.fit(X_train, y_train)
    return clf.score(X_test, y_test)

### Test Cases :

In [ ]:
titanic_train = import_data("./../../../Datasets/Titanic/train.csv");
titanic_test =  import_data("./../../../Datasets/Titanic/test.csv")

In [ ]:
# preprocessing phase 1 for training and validation dataset
titanic_train = data_preprocessor(titanic_train)
print_head(titanic_train)

In [ ]:
titanic_test = data_preprocessor(titanic_test)
print_head(titanic_test)

#### Test Case 1:

In [ ]:
#removed columns with null values
titanic_train_1 = titanic_train.replace({404 : np.nan})
titanic_train_1 = drop_null_attributes(titanic_train_1)
create_logistic_regression_model(titanic_train_1)

#### Test Case 2:

In [ ]:
# removed rows with null values
titanic_train_2 = titanic_train.replace({404 : np.nan})
titanic_train_2 = drop_null_rows(titanic_train_2)
create_logistic_regression_model(titanic_train_2)

#### Test Case 3:

In [ ]:
# replaced null values with mean and mode
titanic_train_3 = titanic_train.replace({404 : np.nan})
titanic_train_3 = replace_nulls_phase_1(titanic_train_3)
create_logistic_regression_model(titanic_train_3)

#### Test Case 4:

In [ ]:
titanic_train_4 = (titanic_train.replace({404 : np.nan})
                                .astype({'Class' : int})
                  )


In [ ]:
# form a dictionary structure of column with nulls as keys and attributes they are most correlated to as a list of values
def get_top_corr_dict(attribute_with_nulls):
    top_corr_dict = {}
    for attribute in attribute_with_nulls:
        top_corr_dict[attribute] = top_correlations(titanic_train_4, attribute)
    return top_corr_dict

In [ ]:
# dataframe of values based on averages columns with null values based on groupby result of previous step
def get_vals_based_on_corr_groups(top_corr_dict, attr_type):
    values_based_corr_dict = {}
    for attr in top_corr_dict.keys():
        if attr_type[attr] == 'category':
            df = titanic_train_4.groupby(top_corr_dict[attr].index.tolist(), as_index= False)[attr].agg(lambda x : x.value_counts().index[0] if len(x.value_counts()) is not 0 else titanic_train_4['Cabin'].value_counts().index[1])
        else:
            df = titanic_train_4.groupby(top_corr_dict[attr].index.tolist(),  as_index= False)[attr].agg(np.mean)
        values_based_corr_dict[attr] = df 
    return values_based_corr_dict

In [ ]:
attribute_with_nulls = get_attributes_with_nulls(titanic_train_4)
top_corr_dict = get_top_corr_dict(attribute_with_nulls)

attr_type = {'Age' : 'discrete', 'Cabin' : 'category', 'Embarked': 'category'}
vals_based_on_corr_groups = get_vals_based_on_corr_groups(top_corr_dict, attr_type)

**Generalize below function update_null_vals for all attributes with null values, code before this function works fine**

In [ ]:
def update_null_vals(row, vals_based_on_corr_groups):
    for attr in vals_based_on_corr_groups.keys():
        if np.isnan(row[attr]):
            if np.isnan(vals_based_on_corr_groups[attr].loc[row['Class']].loc[row['# of Siblings'], 'mean']):
                row['Age'] = main_avg
            else:
                row['Age'] = vals_based_on_corr_groups.loc[row['Class']].loc[row['# of Siblings'], 'mean']
    
    return row

titanic_train_4 = titanic_train_4.apply(update_null_vals, axis = 1, args= (vals_based_on_corr_groups, ));
                         

In [ ]:
titanic_train_4.info()

In [ ]:
top_corr_dict

In [ ]:
vals_based_on_corr_groups['Age'].loc[:,'Class']

#### Future Scope:

- automate preprocessing end to end for all features
- functional groupby for discrete values like "Fare"